# Import

In [1]:
import pyiast
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import time
import os
import scipy.optimize as optim
import matplotlib.pyplot as plt

#df_NAME=pd.read_csv("HEAT_0215.csv")
df_NAME=pd.read_csv("HEAT_0413.csv")


bins_CO2 = []
bins_H2S = []
ccc = 0
for nam in df_NAME["NAME"]:
    print(nam)
    f_tmp = open("iso_"+nam+"_saved.bin",'rb')
    bin_tmp = np.array(pickle.load(f_tmp))
    #print(ccc)
    #print(bin_tmp)

    bins_CO2.append(bin_tmp[[False, True, True]]) ## Pick 1 = CO2, 2 = CH4 

    f_tmp.close()
    ccc = ccc + 1
ccc = 0
summm = 0
for nam in df_NAME["NAME"]:
    f_tmp = open("iso_"+nam+"_saved.bin",'rb') ## 0 = H2S , 1 = CO2, 2 = CH4
    bin_tmp = np.array(pickle.load(f_tmp))
    #print(ccc)
    #print(bin_tmp)
    summm = summm + len(bin_tmp)
    bins_H2S.append(bin_tmp[[True, False, True]]) ## Pick 0 = H2S and 2 = CH4

    f_tmp.close()
    ccc = ccc + 1
    
print('Total isotherm fittings = ',summm )
print('ouf of ', 3*len(df_NAME["NAME"].to_numpy()))
    
Names_CO2=df_NAME["NAME"].to_numpy()
Names_H2S=df_NAME["NAME"].to_numpy()
dH_CO2 = np.array([df_NAME["CO2_Heat"], df_NAME["CH4_Heat"]]).T
dH_H2S = np.array([df_NAME["H2S_Heat"],df_NAME["CH4_Heat"]]).T


IWR-1
LTL-2
MOR-1
SEW-1
Total isotherm fittings =  12
ouf of  12


# Function: iso_mix

In [2]:
Arrh = lambda T,dH ,T_ref: np.exp(-dH/8.3145*(1/T - 1/T_ref)) # Arrhenius equation (Clasius-Clapeyron Equation)

## Isothermal mixture isotherm
def iso_mix(P_par, T, iso_list, dH_list,Tref_list):
    P_norm = []
    for (p,dh,tref) in zip(P_par, dH_list,Tref_list):
        p_n = Arrh(T,dh,tref)*p 
        P_norm.append(p_n)
    P_norm_arr = np.array(P_norm)
    #print(P_norm_mat.T)
    if P_norm_arr.ndim > 1:
        for i in range(len(P_norm[0])):
            p_tmp = P_norm_arr[i,:]
            p_tmp[p_tmp<0.000001] = 0.000001
            q_IAST_tmp = pyiast.iast(p_tmp,
                                     iso_list,
                                     warningoff=True)
    else:
        try:
            p_tmp = P_norm_arr
            p_tmp[p_tmp<0.000001] = 0.000001
            #print(p_tmp)
            q_IAST_tmp = pyiast.iast(p_tmp,
                                    iso_list,
                                     warningoff=True)
        except:    
            try:
                #print('Initial guess error with P = ',P_par)
                x_IG = np.ones(len(p_tmp))/len(p_tmp)
                q_IAST_tmp = pyiast.iast(p_tmp,
                                        iso_list,adsorbed_mole_fraction_guess = x_IG,
                                        warningoff=True)
            except:
                try:
                    arg_min = np.argmin(p_tmp)
                    p_tmp[p_tmp<0.000001] = 0.000001
                    x_IG = 0.05*np.ones(len(p_tmp))
                    x_IG[arg_min] = 1 - 0.05*(len(p_tmp)-1)
                    #print(x_IG)
                    q_IAST_tmp = pyiast.iast(p_tmp,
                                            iso_list,adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)

                except:
                    try:
                        arg_max = np.argmax(p_tmp)
                        p_tmp[p_tmp<0.000001] = 0.000001
                        x_IG = 0.05*np.ones(len(p_tmp))
                        x_IG[arg_max] = 1 - 0.05*(len(p_tmp)-1)
                        #print(x_IG)
                        q_IAST_tmp = pyiast.iast(p_tmp,
                                                iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)        
                    except:
                        try:
                            arg_max = np.argmax(p_tmp)
                            p_tmp[p_tmp<0.000001] = 0.000001
                            x_IG = 0.15*np.ones(len(p_tmp))
                            x_IG[arg_max] = 1 - 0.15*(len(p_tmp)-1)
                            #print(x_IG)
                            q_IAST_tmp = pyiast.iast(p_tmp,
                                                iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)
                        except:
                            try:
                                arg_min = np.argmin(p_tmp)
                                p_tmp[p_tmp<0.000001] = 0.000001
                                x_IG = 0.01*np.ones(len(p_tmp))
                                x_IG[arg_min] = 1 - 0.01*(len(p_tmp)-1)
                                #print(x_IG)
                                q_IAST_tmp = pyiast.iast(p_tmp,
                                            iso_list,adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)

                            except:
                                try:
                                    arg_max = np.argmax(p_tmp)
                                    p_tmp[p_tmp<0.000001] = 0.000001
                                    x_IG = 0.01*np.ones(len(p_tmp))
                                    x_IG[arg_max] = 1 - 0.01*(len(p_tmp)-1)
                                    #print(x_IG)
                                    q_IAST_tmp = pyiast.iast(p_tmp,
                                                    iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)        
                                except:
                                    p_tmp[p_tmp<0.000001] = 0.000001
                                    arg_invalid = p_tmp < 0.00001
                                    arg_valid = (p_tmp < 0.00001) == False
                                    if np.sum(arg_valid) == 1:
                                        q_IAST_tmp = np.zeros(len(iso_list))
                                        q_IAST_tmp[arg_valid] = iso_list[arg_valid].loading(p_tmp[arg_valid])
                                        q_IAST_tmp[arg_invalid] = 0.00001
                                    else:
                                        x_IG = [0.9999, 0.0001]
                                        #print(x_IG)
                                        q_IAST_tmp = pyiast.iast(
                                            p_tmp,iso_list,
                                            adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)
    return q_IAST_tmp

# Pressure points

In [3]:
P_targ = np.array([0.0001,
0.0005,
0.001,
0.01,
0.02,
0.03,
0.04,
0.05,
0.06,
0.07,
0.08,
0.09,
0.1,
0.3,
0.5,
0.7,
0.9,
1,
2,
3,
4,
5,
6,
7,
8,
9,
10])

In [4]:
print(P_targ)

[1.e-04 5.e-04 1.e-03 1.e-02 2.e-02 3.e-02 4.e-02 5.e-02 6.e-02 7.e-02
 8.e-02 9.e-02 1.e-01 3.e-01 5.e-01 7.e-01 9.e-01 1.e+00 2.e+00 3.e+00
 4.e+00 5.e+00 6.e+00 7.e+00 8.e+00 9.e+00 1.e+01]


# Uptake values of CO2 and CH4

## Calculate

In [5]:
y_f_sel = 0.5 ### CO2 mole fraction

q_CO2 = []
q_CH4_0 = []
for isoo,nana,dhdh in zip(bins_CO2, Names_CO2,dH_CO2):
    q_co2_tmp = []
    q_ch4_tmp = []
    for pp in P_targ:
        iast_res = iso_mix(np.array([y_f_sel, 1-y_f_sel])*pp, ## partial pressure
                           343,isoo,dhdh*1000,[298.15,298.15])
        q_co2_tmp.append(iast_res[0])
        q_ch4_tmp.append(iast_res[1])
        
    q_CO2.append(q_co2_tmp)
    q_CH4_0.append(q_ch4_tmp)
    print(nana)
di_co2 = {'P (bar)':P_targ}
di_ch4_0 = {'P (bar)':P_targ}
for qco2, qch4, nam in zip(q_CO2,q_CH4_0,Names_CO2):
    di_co2[nam + ' (mol/kg)'] = qco2
    di_ch4_0[nam+ ' (mol/kg)'] = qch4
df_co2 = pd.DataFrame(di_co2) 
df_ch4_0 = pd.DataFrame(di_ch4_0)    

IWR-1
LTL-2
MOR-1
SEW-1


## Check the data

In [6]:
df_co2

,P (bar),IWR-1 (mol/kg),LTL-2 (mol/kg),MOR-1 (mol/kg),SEW-1 (mol/kg)
0,0.0001,0.000021,0.000017,0.000010,0.000022
1,0.0005,0.000103,0.000085,0.000048,0.000110
2,0.0010,0.000205,0.000169,0.000096,0.000220
3,0.0100,0.002052,0.001693,0.000959,0.002195
4,0.0200,0.004099,0.003382,0.001917,0.004384
5,0.0300,0.006142,0.005068,0.002874,0.006566
6,0.0400,0.008180,0.006752,0.003830,0.008743
7,0.0500,0.010215,0.008432,0.004784,0.010914
8,0.0600,0.012245,0.010109,0.005738,0.013079
9,0.0700,0.014270,0.011783,0.006691,0.015238


In [7]:
df_ch4_0

,P (bar),IWR-1 (mol/kg),LTL-2 (mol/kg),MOR-1 (mol/kg),SEW-1 (mol/kg)
0,0.0001,0.000009,0.000004,0.000010,0.000010
1,0.0005,0.000047,0.000020,0.000048,0.000051
2,0.0010,0.000094,0.000041,0.000096,0.000102
3,0.0100,0.000943,0.000408,0.000961,0.001020
4,0.0200,0.001885,0.000816,0.001920,0.002038
5,0.0300,0.002824,0.001223,0.002879,0.003054
6,0.0400,0.003763,0.001629,0.003836,0.004069
7,0.0500,0.004700,0.002035,0.004793,0.005081
8,0.0600,0.005635,0.002440,0.005748,0.006092
9,0.0700,0.006569,0.002844,0.006703,0.007101


In [8]:
df_co2.to_csv('qCO2_IAST_CO2_CASE.csv',index = False)
df_ch4_0.to_csv('qCH4_IAST_CO2_CASE.csv',index = False)

# Uptake values of H2S and CH4

In [11]:
print(bins_H2S[0][1].params)
print(bins_H2S[1][1].params)

{'M': 1.6975009784578017, 'Ka': 0.2526768146211568, 'Kb': 0.019718975140124044}
{'M': 3.168853808082409, 'K': 0.05480173932773619}


## Calculate

In [15]:
print(q_H2S[0][-1])
print(q_H2S[1][-1])
print(q_H2S[2][-1])

0.9295061975070579
0.9295061975070579
0.9295061975070579


In [21]:
y_f_sel = 0.5 ### CO2 mole fraction

q_H2S = []
q_CH4_1 = []
for isoo,nana,dhdh in zip(bins_H2S, Names_H2S,dH_H2S):
    q_h2s_tmp = []
    q_ch4_tmp = []
    for pp in P_targ:
        iast_res = iso_mix(np.array([y_f_sel, 1-y_f_sel])*pp, ## partial pressure
                           343,isoo,dhdh*1000,[298.15,298.15])
        q_h2s_tmp.append(iast_res[0])
        q_ch4_tmp.append(iast_res[1])
    q_H2S.append(q_h2s_tmp)
    q_CH4_1.append(q_ch4_tmp)
    print(nana)
di_h2s = {'P (bar)':P_targ}
di_ch4_1 = {'P (bar)':P_targ}
for qh2s, qch4, nam in zip(q_H2S,q_CH4_1,Names_H2S):
    di_h2s[nam + ' (mol/kg)'] = qh2s
    di_ch4_1[nam+ ' (mol/kg)'] = qch4
df_h2s = pd.DataFrame(di_h2s) 
df_ch4_1 = pd.DataFrame(di_ch4_1)    

IWR-1
LTL-2
MOR-1
SEW-1


## Check the data

In [22]:
df_h2s

,P (bar),IWR-1 (mol/kg),LTL-2 (mol/kg),MOR-1 (mol/kg),SEW-1 (mol/kg)
0,0.0001,0.000018,0.000017,0.000022,0.000019
1,0.0005,0.000091,0.000083,0.000109,0.000094
2,0.0010,0.000181,0.000165,0.000217,0.000187
3,0.0100,0.001814,0.001654,0.002171,0.001873
4,0.0200,0.003625,0.003307,0.004336,0.003742
5,0.0300,0.005435,0.004959,0.006494,0.005607
6,0.0400,0.007242,0.006610,0.008646,0.007470
7,0.0500,0.009046,0.008261,0.010792,0.009328
8,0.0600,0.010848,0.009911,0.012930,0.011184
9,0.0700,0.012648,0.011560,0.015063,0.013036


In [18]:
df_ch4_1

,P (bar),IWR-1 (mol/kg),LTL-2 (mol/kg),MOR-1 (mol/kg),SEW-1 (mol/kg)
0,0.0001,0.000009,0.000004,0.000010,0.000010
1,0.0005,0.000047,0.000020,0.000048,0.000051
2,0.0010,0.000094,0.000041,0.000096,0.000102
3,0.0100,0.000943,0.000408,0.000960,0.001020
4,0.0200,0.001885,0.000816,0.001918,0.002039
5,0.0300,0.002826,0.001224,0.002873,0.003057
6,0.0400,0.003766,0.001631,0.003827,0.004073
7,0.0500,0.004704,0.002038,0.004778,0.005087
8,0.0600,0.005642,0.002444,0.005727,0.006100
9,0.0700,0.006578,0.002850,0.006674,0.007112


In [20]:
df_h2s.to_csv('qH2S_IAST_H2S_CASE.csv',index = False)
df_ch4_1.to_csv('qCH4_IAST_H2S_CASE.csv',index = False)